# BTC Price Analysis
This notebook demonstrates how to visualize the Bitcoin price, the Fear & Greed index, and the predicted price from our simple linear regression model.

In [ ]:
# Import required libraries
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


## Load the processed data
We will load the processed Bitcoin price and Fear & Greed index data from a Parquet file.

In [ ]:
# Load the processed data
df = pl.read_parquet('../data/processed/fear_and_greed_history_5min.parquet')
df.head()


## Prepare features and target
We will use the Fear & Greed index as the feature and the BTC price as the target for our linear regression model. Rows with missing values will be dropped.

In [ ]:
# Use correct column names: 'avg_actual_value' for Fear & Greed, 'close_bitcoin_price_usd' for BTC price
df = df.drop_nulls(['avg_actual_value', 'close_bitcoin_price_usd'])
X = df.select('avg_actual_value').to_numpy()
y = df.select('close_bitcoin_price_usd').to_numpy().flatten()


## Fit the linear regression model
We will fit a simple linear regression model to predict BTC price from the Fear & Greed index.

In [ ]:
# Fit linear regression model (same as train.py)
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)


## Visualize actual vs predicted BTC prices
We will plot the actual BTC prices and the predicted prices from our linear regression model to compare their trends over time.

In [ ]:
# Plot Fear & Greed index, actual BTC price, and predicted BTC price
import matplotlib.dates as mdates

interval_times = df.select('interval_end_time').to_numpy().flatten()
interval_times = np.array(interval_times, dtype='datetime64[ms]')
fear_greed = df.select('avg_actual_value').to_numpy().flatten()

fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Interval End Time')
ax1.set_ylabel('BTC Price', color=color)
ax1.plot(interval_times, y, label='Actual BTC Price', color='tab:blue')
ax1.plot(interval_times, y_pred, label='Predicted BTC Price', color='tab:orange', linestyle='--')
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Fear & Greed Index', color=color)
ax2.plot(interval_times, fear_greed, label='Fear & Greed Index', color=color, alpha=0.5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')

fig.autofmt_xdate()
plt.title('BTC Price and Fear & Greed Index with Linear Regression Prediction')
plt.tight_layout()
plt.show()
